<a href="https://colab.research.google.com/github/sarwaridas/AdversarialPatches/blob/main/src/resnet_cifar10_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Set up drive location 

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Mounted at /gdrive


In [ ]:
%cd /gdrive/My Drive/Final Project/

/gdrive/My Drive/Final Project


### Import Required Packages

In [ ]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

### Defining our Resnet model

In [ ]:
# define the Resnet model
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.identity = nn.Sequential()
        if stride != 1:
                self.identity = nn.Sequential(
                     nn.Conv2d(in_channels,out_channels, kernel_size=1, stride=stride),
                     nn.BatchNorm2d(out_channels)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        residual = self.identity(x)
        out += residual
        out = F.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self.layer(16, stride=1)
        self.layer2 = self.layer(32, stride=2)
        self.layer3 = self.layer(64, stride=2)
        self.linear = nn.Linear(64, 10)

    def layer(self, out_channels, stride):
      strides = [stride] + [1]*(2)
      layers = []
      for stride in strides:
            layers.append(Block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
      return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

### Setting up train and test data loaders and preprocessing functions

In [ ]:
# useful libraries
import torchvision
import torchvision.transforms as transforms

#############################################
# specify preprocessing function

#Combining 1) random cropping with a padding of 4 and 2) random flipping

size, padding = 32, 4

transform_train = transforms.Compose(
    [transforms.RandomCrop(size, padding), transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_val = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
#############################################

In [ ]:
DATA_ROOT = "./data"

batch_size = 512

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_val)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
#############################################

Files already downloaded and verified
Files already downloaded and verified


### Initializing the model

In [ ]:
# specify the device for computation
#############################################

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = ResNet()
net.to(device)

#############################################

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Block(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (identity): Sequential()
    )
    (1): Block(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (identity): Sequential()
    )
 

### Setting up the training process.

In [ ]:
import torch.nn as nn
import torch.optim as optim

# # momentum for optimizer
MOMENTUM = 0.9

#############################################
# create loss function
criterion =  nn.CrossEntropyLoss()

# Add optimizer
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=MOMENTUM,weight_decay=1e-4)
#############################################

In [ ]:
# total number of training epochs
EPOCHS = 100

# the folder where the trained model is saved
CHECKPOINT_FOLDER = "./saved_model"

# start the training/validation process

best_val_acc = 0
current_learning_rate = 0.1
DECAY_EPOCHS= [90,120]
DECAY= 0.1
grad_clip=0.1

print("==> Training starts!")
print("="*50)
for i in range(0, EPOCHS):

    #handle the learning rate scheduler.
    if i in DECAY_EPOCHS and i != 0:
        current_learning_rate = current_learning_rate * DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
    
    ######################
    # switch to train mode
    net.train(True)
    #######################
    print("Epoch %d:" %i)
    # this help you compute the training accuracy
    total_examples = 0
    correct_examples = 0
    train_loss = 0 # track training loss if you want
    
    # Train the model for 1 epoch.
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        ####################################
        # copy inputs to device
        inputs, targets = inputs.to(device), targets.to(device)
        
        # compute the output and loss
        outputs=net(inputs)
        loss = criterion(outputs, targets)

        # zero the gradient
        optimizer.zero_grad()
    
        # backpropagation
        loss.backward()
        
        if grad_clip is not None: 
                nn.utils.clip_grad_value_(net.parameters(), grad_clip)
        #apply gradient and update the weights
        optimizer.step()

        # count the number of correctly predicted samples in the current batch
        _, predicted = torch.max(outputs, 1)
        total_examples += targets.size(0)
        correct_examples += (predicted == targets).sum().item()
        train_loss+= loss
        ####################################
              
    avg_loss = train_loss / len(trainloader)
    avg_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))
    
    # Validate on the validation dataset
    #######################
    # switch to eval mode
    net.eval()
    
    #######################

    # this help you compute the validation accuracy
    total_examples = 0
    correct_examples = 0
    
    val_loss = 0 # again, track the validation loss if you want

    # disable gradient during validation, which can save GPU memory
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            ####################################
            # copy inputs to device
            inputs, targets = inputs.to(device), targets.to(device)
            
            # compute the output and loss
            outputs=net(inputs)
            loss = criterion(outputs, targets)
            
            # count the number of correctly predicted samples in the current batch
            _, predicted = torch.max(outputs, 1)
            total_examples += targets.size(0)
            correct_examples += (predicted == targets).sum().item()
            val_loss+= loss
            ####################################

    avg_loss = val_loss / len(testloader)
    avg_acc = correct_examples / total_examples
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))
    # scheduler.step()  

    # save the model checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        torch.save(net.state_dict(), "resnet_91_98.ckpt")

        
    print('')

print("="*50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")